In [ ]:
import pandas
from pony import orm
import jinja2
import db
import os
import mimetypes
import subprocess
import getpass
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
db.use_db()

In [ ]:
def email_from_template(change=None):
    sub = db.Submission[state.submission_id]

    results = [
     {'name': i.problem.name,
      'score': orm.sum(i.feedback.score),
      'max_score' : orm.max(i.problem.feedback_options.score, default=0),
      'feedback': [{'short': fo.text, 
                    'score': fo.score, 
                    'description': fo.description} for fo in i.feedback],
      'remarks': i.remarks,
     } for i in sub.solutions]

    student = sub.student.to_dict()
    student['total'] = sum(i['score'] for i in results)

    rendered_email = template.render(student=student, results=results)
    email_result.value = '<pre>' + rendered_email + '</pre>'
    return rendered_email


In [ ]:
template = jinja2.Template("""Dear {{student.first_name.split(' ') | first }} {{student.last_name}},

Below please find attached the scans of your exam and our feedback.
If you have any questions, don't hesitate to contant us during the exercise classes 
or in the course chat https://chat.quantumtinkerer.tudelft.nl/solidstate

Additionally, you may want to compare your results with the overall performance
of your colleagues over here: http://nbviewer.jupyter.org/url/antonakhmerov.org/misc/minitest_3_summary.ipynb
as well as a reference solution http:antonakhmerov.org/misc/http://antonakhmerov.org/misc/minitest3_Jasper.pdf

{% for problem in results | sort(attribute='name') if problem.feedback  -%}
{{problem.name}} (your score: {{problem.score}} out of {{problem.max_score}}):
{% for feedback in problem.feedback %}
    * {{ (feedback.description or feedback.short) | wordwrap | indent(width=6) }}
{% endfor %}
{%- if problem.remarks %}
    * {{ problem.remarks | wordwrap | indent(width=6) }}
{% endif %}
{% endfor %}

{%- if student.total < 60 -%}
As you also see in the overall results, this test was hard not only for you,
but also for many of your colleagues. Its important difference from the first
two tests is the amount of calculations you need to perform. Most of the
course is like that, so if you find it problematic: exercise and also approach
us for advice if something is unclear.
{%- elif student.total < 80 -%}
You have passed this test, which likely means you can pass the final exam if
you continue the same way. There is however a lot of room for improvement.
{%- else -%}
You are one of the few who did not encounter a lot of trouble with this
test. Congratulations!
{%- endif%}

Best,
Solid state course team.
""")

template.trim_blocks = template.lstrip_blocks = True
jinja2.environment.trim_blocks = jinja2.environment.lstrip_blocks = True

messages = []
with orm.db_session:
    for sub in db.Exam[2].submissions:
        filename = '{}.pdf'.format(sub.student.id)
        subprocess.call(['convert', *sorted(sub.pages.path), filename])
        results = [
         {'name': i.problem.name,
          'score': orm.sum(i.feedback.score),
          'max_score' : orm.max(i.problem.feedback_options.score, default=0),
          'feedback': [{'short': fo.text, 
                        'score': fo.score, 
                        'description': fo.description} for fo in i.feedback],
          'remarks': i.remarks,
         } for i in sub.solutions]

        student = sub.student.to_dict()
        student['total'] = sum(i['score'] for i in results)

        msg = MIMEMultipart()
        msg['Subject'] = 'Your results for the third minitest'
        msg['From'] = 'no-reply@solidstate.tudelft.nl'
        msg['To'] = sub.student.email
        msg['Reply-to'] = 'no-reply@solidstate.tudelft.nl'
        msg.attach(MIMEText(template.render(student=student, results=results), 'plain'))

        ctype, encoding = mimetypes.guess_type(filename)
        if ctype is None or encoding is not None:
            ctype = 'application/octet-stream'
        maintype, subtype = ctype.split('/', 1)
        with open(filename, 'rb') as fp:
            pdf = MIMEBase(maintype, subtype)
            pdf.set_payload(fp.read())
        encoders.encode_base64(pdf)
        # Set the filename parameter
        pdf.add_header('Content-Disposition', 'attachment', filename=filename)
        msg.attach(pdf)
        os.remove(filename)
        messages.append(msg)

In [ ]:
password = getpass.getpass()

In [ ]:
with smtplib.SMTP_SSL('smtp-a.tudelft.nl', 465) as s:
    s.login('aakhmerov', password)
    for number, msg in enumerate(messages):
        print(number)
        s.sendmail('a.r.akhmerov@tudelft.nl', [msg['To']],
                   msg.as_string())
